# Key Levels Detection (Close)

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import time
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import requests

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120

import warnings
warnings.filterwarnings('ignore')

In [5]:
entrade_headers = {
        'authority': 'services.entrade.com.vn',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-US,en;q=0.9',
        'dnt': '1',
        'origin': 'https://banggia.dnse.com.vn',
        'referer': 'https://banggia.dnse.com.vn/',
        'sec-ch-ua': '"Edge";v="114", "Chromium";v="114", "Not=A?Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'cross-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1788.0'
    }
DNSE_DATA_HISTORY_URL = 'https://services.entrade.com.vn/chart-api/v2/ohlcs/derivative'

In [6]:
def getStockHistoryData(ticker, timestamp_from=0, timestamp_to=0):
    if timestamp_from == 0:
        three_months = date.today() + relativedelta(months=-3)
        timestamp_from = datetime.strptime(three_months.strftime("%m/%d/%Y") + ', 00:00:0', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()
    if timestamp_to == 0:
        timestamp_to = datetime.strptime(date.today().strftime("%m/%d/%Y") + ', 23:59:00', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()

    params = {
        "resolution": "5",
        "symbol": "VN30F1M",
        "from": int(timestamp_from),
        "to": int(timestamp_to)
    }

    x = requests.get(DNSE_DATA_HISTORY_URL, params=params, headers=entrade_headers)
    response = x.json()

    import numpy as np
    import pandas as pd

    timestamp = np.array(response['t']).astype(int)
    close = np.array(response['c']).astype(float)
    open = np.array(response['o']).astype(float)
    high = np.array(response['h']).astype(float)
    low = np.array(response['l']).astype(float)
    volume = np.array(response['v']).astype(int)

    dataset = pd.DataFrame({'Time': timestamp, 'Open': list(open), 'High': list(high), 'Low': list(low),
                            'Close': list(close), 'Volume': list(volume)},
                           columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    return dataset
def prepareData(htd):
    if 'Time' in htd.columns:
        from datetime import datetime

        htd['DateStr'] = htd.apply(
            lambda x: datetime.fromtimestamp(x['Time']).strftime("%Y-%m-%d %H:%M:%S"), axis=1)

    htd['Date'] = pd.to_datetime(htd['DateStr'])
    # htd['Date'] = htd['Date'] + pd.DateOffset(hours=7)
    ticker_data = htd.set_index('Date')
    ticker_data.drop(columns=['Time', 'DateStr'], inplace=True)
    return ticker_data

## Load price data from DNSE

In [7]:
ticker = "VN30F1M"
htd = getStockHistoryData(ticker, 1, 0)
ticker_data = prepareData(htd)
dataset = ticker_data.dropna()
dataset

,Open,High,Low,Close,Volume
Date,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765
...,...,...,...,...,...
2024-07-03 14:15:00,1306.0,1308.0,1306.0,1307.0,7539
2024-07-03 14:20:00,1306.8,1307.1,1305.8,1306.5,5894
2024-07-03 14:25:00,1306.6,1306.7,1305.1,1305.5,6427


In [8]:
data = dataset.copy()
data['max_5bars_prev'] = data['Close'].rolling(5).max()
data['max_5bars_next'] = data['Close'].shift(-4).rolling(5).max()
data['min_5bars_prev'] = data['Close'].rolling(5).min()
data['min_5bars_next'] = data['Close'].shift(-4).rolling(5).min()
data

,Open,High,Low,Close,Volume,max_5bars_prev,max_5bars_next,min_5bars_prev,min_5bars_next
Date,,,,,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812,NaN,NaN,NaN,NaN
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323,NaN,NaN,NaN,NaN
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207,NaN,NaN,NaN,NaN
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196,NaN,NaN,NaN,NaN
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765,943.7,945.3,942.6,943.7
...,...,...,...,...,...,...,...,...,...
2024-07-03 14:15:00,1306.0,1308.0,1306.0,1307.0,7539,1308.4,1307.0,1305.1,1305.5
2024-07-03 14:20:00,1306.8,1307.1,1305.8,1306.5,5894,1308.4,NaN,1306.0,NaN
2024-07-03 14:25:00,1306.6,1306.7,1305.1,1305.5,6427,1307.0,NaN,1305.5,NaN


In [9]:
data['is_r_keylevel'] = data.apply(lambda r: True if (r['Close'] == r['max_5bars_prev'] and r['Close'] == r['max_5bars_next']) else False, axis=1)
data['is_s_keylevel'] = data.apply(lambda r: True if (r['Close'] == r['min_5bars_prev'] and r['Close'] == r['min_5bars_next']) else False, axis=1)
data

,Open,High,Low,Close,Volume,max_5bars_prev,max_5bars_next,min_5bars_prev,min_5bars_next,is_r_keylevel,is_s_keylevel
Date,,,,,,,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812,NaN,NaN,NaN,NaN,False,False
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323,NaN,NaN,NaN,NaN,False,False
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207,NaN,NaN,NaN,NaN,False,False
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196,NaN,NaN,NaN,NaN,False,False
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765,943.7,945.3,942.6,943.7,False,False
...,...,...,...,...,...,...,...,...,...,...,...
2024-07-03 14:15:00,1306.0,1308.0,1306.0,1307.0,7539,1308.4,1307.0,1305.1,1305.5,False,False
2024-07-03 14:20:00,1306.8,1307.1,1305.8,1306.5,5894,1308.4,NaN,1306.0,NaN,False,False
2024-07-03 14:25:00,1306.6,1306.7,1305.1,1305.5,6427,1307.0,NaN,1305.5,NaN,False,False


In [10]:
data[data['is_r_keylevel'] == True][['is_r_keylevel']].tail(10)

,is_r_keylevel
Date,
2024-07-02 09:05:00,True
2024-07-02 09:10:00,True
2024-07-02 10:15:00,True
2024-07-02 11:05:00,True
2024-07-02 13:05:00,True
2024-07-02 13:40:00,True
2024-07-03 09:10:00,True
2024-07-03 09:40:00,True
2024-07-03 10:45:00,True


In [11]:
data[data['is_s_keylevel'] == True]

,Open,High,Low,Close,Volume,max_5bars_prev,max_5bars_next,min_5bars_prev,min_5bars_next,is_r_keylevel,is_s_keylevel
Date,,,,,,,,,,,
2018-08-13 10:50:00,943.6,943.8,942.3,943.2,1541,944.8,946.1,943.2,943.2,False,True
2018-08-13 13:25:00,947.7,947.7,946.7,947.0,2614,948.7,948.8,947.0,947.0,False,True
2018-08-13 13:50:00,948.4,948.6,947.4,947.7,1308,948.8,952.0,947.7,947.7,False,True
2018-08-14 10:00:00,955.8,956.0,955.4,955.4,901,956.3,957.1,955.4,955.4,False,True
2018-08-14 10:40:00,953.4,953.7,952.6,952.9,2550,956.1,954.1,952.9,952.9,False,True
...,...,...,...,...,...,...,...,...,...,...,...
2024-07-02 14:45:00,1295.2,1295.2,1295.2,1295.2,7069,1296.9,1297.8,1295.2,1295.2,False,True
2024-07-03 10:15:00,1295.4,1295.4,1294.5,1294.9,3723,1296.3,1296.3,1294.9,1294.9,False,True
2024-07-03 10:20:00,1294.9,1294.9,1294.5,1294.9,1640,1295.6,1296.3,1294.9,1294.9,False,True


In [12]:
has_keylevel = data[(data['is_r_keylevel'] == True) | (data['is_s_keylevel'] == True)]
has_keylevel['count'] = '1'
has_keylevel['keylevels'] = has_keylevel['Close']
has_keylevel

,Open,High,Low,Close,Volume,max_5bars_prev,max_5bars_next,min_5bars_prev,min_5bars_next,is_r_keylevel,is_s_keylevel,count,keylevels
Date,,,,,,,,,,,,,
2018-08-13 09:25:00,943.8,945.9,943.7,945.3,2469,945.3,945.3,942.6,943.7,True,False,1,945.3
2018-08-13 09:50:00,945.0,946.0,944.3,946.0,1213,946.0,946.0,943.7,945.0,True,False,1,946.0
2018-08-13 09:55:00,946.2,946.4,945.6,946.0,1873,946.0,946.0,943.7,945.0,True,False,1,946.0
2018-08-13 10:50:00,943.6,943.8,942.3,943.2,1541,944.8,946.1,943.2,943.2,False,True,1,943.2
2018-08-13 13:10:00,948.0,948.7,947.3,948.7,1230,948.7,948.7,947.0,947.0,True,False,1,948.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-03 10:20:00,1294.9,1294.9,1294.5,1294.9,1640,1295.6,1296.3,1294.9,1294.9,False,True,1,1294.9
2024-07-03 10:45:00,1296.2,1297.3,1295.8,1297.1,2984,1297.1,1297.1,1295.4,1296.4,True,False,1,1297.1
2024-07-03 11:10:00,1296.9,1296.9,1296.0,1296.2,1548,1296.9,1297.7,1296.2,1296.2,False,True,1,1296.2


In [21]:
has_keylevel[(has_keylevel.index > '2024-07-02 00:00:00') & (has_keylevel.index < '2024-07-02 15:00:00')]

,Open,High,Low,Close,Volume,max_5bars_prev,max_5bars_next,min_5bars_prev,min_5bars_next,is_r_keylevel,is_s_keylevel,count,keylevels
Date,,,,,,,,,,,,,
2024-07-02 09:05:00,1286.7,1287.4,1286.6,1287.4,4444,1287.4,1287.4,1284.8,1285.1,True,False,1,1287.4
2024-07-02 09:10:00,1287.3,1287.6,1287.1,1287.4,1916,1287.4,1287.4,1284.8,1285.1,True,False,1,1287.4
2024-07-02 09:25:00,1286.4,1286.6,1284.9,1285.1,4732,1287.4,1286.5,1285.1,1285.1,False,True,1,1285.1
2024-07-02 09:50:00,1285.3,1285.4,1284.6,1285.2,3318,1286.5,1287.5,1285.2,1285.2,False,True,1,1285.2
2024-07-02 10:15:00,1286.3,1291.5,1286.2,1290.8,10276,1290.8,1290.8,1286.1,1290.1,True,False,1,1290.8
2024-07-02 11:05:00,1294.4,1296.6,1294.2,1296.4,9719,1296.4,1296.4,1291.2,1294.1,True,False,1,1296.4
2024-07-02 13:05:00,1294.8,1296.8,1294.7,1295.7,4098,1295.7,1295.7,1294.3,1294.6,True,False,1,1295.7
2024-07-02 13:15:00,1295.0,1295.3,1294.4,1294.6,2722,1295.7,1295.5,1294.6,1294.6,False,True,1,1294.6
2024-07-02 13:40:00,1295.0,1299.0,1294.9,1298.3,7441,1298.3,1298.3,1295.0,1296.9,True,False,1,1298.3


In [14]:
def cal_keylevels(tick):
    if tick.empty:
        return ''
    keylevels = []
    tick = tick.sort_values(ascending=False)
    i = 0
    while i < len(tick):
        if i == 0 or i == len(tick)-1:
            keylevels.append(tick[i])
        else:
            if tick[i-1] > tick[i] + 3 or tick[i] > tick[i+1] + 3:
                keylevels.append(tick[i])
        i = i+1
    return ", ".join(map(str, keylevels))

has_keylevel_day = has_keylevel.resample("D").agg({
        'count': 'count',
        'keylevels': cal_keylevels
        #'keylevels': lambda x: ", ".join(map(str, x))
    })
has_keylevel_day = has_keylevel_day[has_keylevel_day['count'] != 0]

In [15]:
has_keylevel_day[['count']]

,count
Date,
2018-08-13,7
2018-08-14,7
2018-08-15,10
2018-08-16,10
2018-08-17,7
...,...
2024-06-27,11
2024-06-28,12
2024-07-01,8


In [16]:
has_keylevel_day[['count']].min()

count    2
dtype: int64

In [17]:
has_keylevel_day['count'].max()

21

In [18]:
has_keylevel_day[['keylevels']]

,keylevels
Date,
2018-08-13,"948.7, 943.2"
2018-08-14,"961.2, 957.1, 952.3"
2018-08-15,"962.1, 955.2"
2018-08-16,"944.3, 936.2"
2018-08-17,"954.3, 946.5"
...,...
2024-06-27,"1290.3, 1281.4"
2024-06-28,"1292.0, 1286.5, 1283.0, 1274.3"
2024-07-01,"1275.8, 1272.1"


In [19]:
has_keylevel_day['keylevels'] = has_keylevel_day['keylevels'].shift(1)
join_data = has_keylevel_day[['keylevels']]

In [20]:
data2 = dataset.copy()
data2['cs'] = data2.apply(lambda r: get_type_candlestick(r), axis=1)
data2['cs_shift_1'] = data2['cs'].shift(1)
data2['cs_shift_2'] = data2['cs'].shift(2)
data2['cs_reversal'] = data2.apply(lambda r: has_reversal_pattern(r), axis=1)
data2["ema_line"] = ta.ema(data2["Close"], length=20)
data2['above_ma'] = data2.apply(lambda r: 1 if r['Close'] > r['ema_line'] else 0, axis=1)
data2['below_ma'] = data2.apply(lambda r: 1 if r['Close'] < r['ema_line'] else 0, axis=1)
data2['total_above_ma'] = data2['above_ma'].rolling(150).sum()
data2['total_below_ma'] = data2['below_ma'].rolling(150).sum()
data2['trend'] = data2.apply(lambda r: 'switch' if r['total_above_ma'] == r['total_below_ma'] else ('up' if r['total_above_ma'] > r['total_below_ma'] else 'down'), axis=1)

NameError: name 'get_type_candlestick' is not defined

In [ ]:
data2 = data2.assign(time_d=pd.PeriodIndex(data2.index, freq='1D').to_timestamp())
data2 = pd.merge(data2, join_data, left_on="time_d", right_index=True, how="left")

In [ ]:
data2 = data2[data2['keylevels'].notnull()]
data2

In [ ]:
def cross_keylevel(row):
    cross = ''
    keylevels = str(row['keylevels']).split(", ")
    current_price = row['Close']
    for keylevel in keylevels:
        if row['High'] > float(keylevel) > row['Low']:
            cross = 'cross'
    return cross
data2['cross'] = data2.apply(lambda r: cross_keylevel(r), axis=1)

In [ ]:
data2

In [ ]:
kz = data2.iloc[-1]
keylevels = str(kz['keylevels']).split(", ")
keylevels = [float(i) for i in keylevels]
keylevels.sort(reverse=True)
keylevels

In [ ]:
def cal_signal(row):
    signal = ''
    if row['trend'] == 'down':
        keylevels = str(row['keylevels']).split(", ")
        keylevels = [float(i) for i in keylevels]
        max_keylevel = keylevels[0]
        if row['Open'] > row['Close'] and row['High'] > max_keylevel > row['Close']:
            signal = 'short'
        current_price = row['Close']
        for keylevel in keylevels:
            if row['Open'] > row['Close'] and row['High'] > float(keylevel) > row['Close']:
                # Cross key_level and black candlestick
                if row['cs_reversal'] == 1:
                    signal = 'short'
    elif row['trend'] == 'up':
        signal = ''
        # keylevels = str(row['keylevels']).split(", ")
        # current_price = row['Close']
        # for keylevel in keylevels:
        #     if row['Open'] < row['Close'] and row['Low'] < float(keylevel) < row['Close']:
        #         signal = 'long'
    return signal
data2['signal'] = data2.apply(lambda r: cal_signal(r), axis=1)

In [ ]:
selected_day = data2[(data2.index > '2024-06-07 00:00:00') & (data2.index < '2024-06-07 15:00:00')]
selected_day[selected_day['cross'] == 'cross']

In [ ]:
selected_day.iloc[-1]['keylevels']

In [ ]:
data2[data2['signal'] == 'short']